<a href="https://colab.research.google.com/github/nasibehmohammadi/Thesis/blob/main/1lstm_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 1️⃣ بارگذاری دیتاست شخصی (لطفاً نام فایل را تغییر بده!)
file_path = "/content/kaggle2.csv"  # نام فایل خود را اینجا بگذار
df = pd.read_csv(file_path)

# 2️⃣ بررسی ساختار داده‌ها
print("🔍 بررسی ۵ سطر اول دیتاست:")
print(df.head())

# 3️⃣ مشخص کردن ویژگی‌های ورودی و خروجی
# نام ستون‌ها را بر اساس دیتاست خود تغییر بده!
feature_columns = ["B","C","D","E","F","G","H","I","J"]  # نام ویژگی‌ها
target_column = "A"  # نام ستون هدف (برچسب‌ها)

# تبدیل دیتافریم به آرایه NumPy
X_data = df[feature_columns].values
y_data = df[target_column].values

# مشخص کردن تعداد ویژگی‌ها
num_features = X_data.shape[1]

# 4️⃣ تنظیم داده‌ها برای مدل LSTM
time_steps = 10  # تعداد مراحل زمانی (مثلاً ۱۰ ورود اخیر هر کاربر)

# ایجاد دنباله‌های زمانی برای LSTM
X_sequences = []
y_sequences = []

for i in range(len(X_data) - time_steps):
    X_sequences.append(X_data[i : i + time_steps])  # داده‌های ۱۰ زمان اخیر
    y_sequences.append(y_data[i + time_steps])  # مقدار هدف بعد از ۱۰ مرحله

X_sequences = np.array(X_sequences)
y_sequences = np.array(y_sequences)

# 5️⃣ تقسیم داده‌ها به مجموعه آموزش و تست
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.2, random_state=42)

# 6️⃣ مقیاس‌بندی داده‌ها
scaler = MinMaxScaler()
X_train = X_train.reshape(-1, num_features)
X_test = X_test.reshape(-1, num_features)

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = X_train.reshape(-1, time_steps, num_features)
X_test = X_test.reshape(-1, time_steps, num_features)

# 7️⃣ ایجاد مدل LSTM
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(time_steps, num_features)),
    Dropout(0.2),
    LSTM(32, return_sequences=False),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

# 8️⃣ کامپایل مدل
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 9️⃣ آموزش مدل روی دیتای شخصی
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# 🔟 ارزیابی مدل
loss, accuracy = model.evaluate(X_test, y_test)
print(f"✅ دقت مدل روی داده‌های تست: {accuracy * 100:.2f}%")

# ۱۱️⃣ پیش‌بینی روی یک نمونه جدید
test_sample = X_test[0].reshape(1, time_steps, num_features)
prediction = model.predict(test_sample)

if prediction > 0.5:
    print("🚨 ورود مشکوک! نیاز به احراز هویت بیشتر.")
else:
    print("✅ ورود ایمن است.")


🔍 بررسی ۵ سطر اول دیتاست:
   A      B      C       D       E       F       G       H       I       J
0  1  39353  85475  117961  118300  123472  117905  117906  290919  117908
1  1  17183   1540  117961  118343  123125  118536  118536  308574  118539
2  1  36724  14457  118219  118220  117884  117879  267952   19721  117880
3  1  36135   5396  117961  118343  119993  118321  240983  290919  118322
4  1  42680   5905  117929  117930  119569  119323  123932   19793  119325


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 19s 16ms/step - accuracy: 0.9421 - loss: 0.2481 - val_accuracy: 0.9435 - val_loss: 0.2178
Epoch 2/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 13s 15ms/step - accuracy: 0.9445 - loss: 0.2174 - val_accuracy: 0.9435 - val_loss: 0.2173
Epoch 3/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - accuracy: 0.9432 - loss: 0.2208 - val_accuracy: 0.9435 - val_loss: 0.2173
Epoch 4/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 21s 16ms/step - accuracy: 0.9405 - loss: 0.2284 - val_accuracy: 0.9435 - val_loss: 0.2178
Epoch 5/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - accuracy: 0.9414 - loss: 0.2252 - val_accuracy: 0.9435 - val_loss: 0.2172
Epoch 6/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 21s 16ms/step - accuracy: 0.9401 - loss: 0.2290 - val_accuracy: 0.9435 - val_loss: 0.2172
Epoch 7/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - accuracy: 0.9414 - loss: 0.2240 - val_accuracy: 0.9435 - val_loss: 0.2191
Epoch 8/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - accuracy: 0.9412 - loss: 0.2255 - 